### Alvins yrken för alla personer 

Läser alla personer från Alvin via webscrabing 
* Wikidata [P6821](https://www.wikidata.org/wiki/Property_talk:P6821)
* [T225522](https://phabricator.wikimedia.org/T225522) "Property Alvin P6821 created and need to be matched"

* GITHUB [salgo60/HISCOKoder](https://github.com/salgo60/HISCOKoder)
  * this [Notebook](https://github.com/salgo60/HISCOKoder/blob/main/Jupyter/Alvin%20yrken%20version%202.ipynb)

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-08-15 23:48:17.662016


In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup, NavigableString, Tag
from tqdm.notebook import tqdm  

newList = []
urlBase = "https://www.alvin-portal.org/alvin/view.jsf?pid="
#for index, row in tqdm(total=WDAlvintot.shape[0]):    
for i in tqdm(range(83000)):
    alvinid = "alvin-person:" + str(i)
    url = urlBase + alvinid
#    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    for header in soup.find_all('h2'): 
        nextNode = header
        if "Occupation" in header:
            while True:
            
                nextNode = nextNode.nextSibling
                if nextNode is None:
                    break
                if isinstance(nextNode, NavigableString):
#                    print (alvinid,"-","\tyrke: ",nextNode.strip())
#                    print (alvinid,"-",person,"\tyrke: ",nextNode.strip())
#alvin-person:1063 - Marie Curie 	yrke:  Fysiker, Radiolog, Kemist, Chemist, Physicist, kemist, fysiker
#alvin-person:48342 - Simone de Beauvoir 	yrke:  författare, filosof, feminist
#alvin-person:27012 - Ingmar Bergman 	yrke:  Regissör, Författare, regissör
#alvin-person:64330 - Grazia Deledda 	yrke:  Författarinna
#
#
                    newList.append(nextNode.strip())
                    break

print("Lines found:\t", len(newList))    
# TBD
    

  0%|          | 0/83000 [00:00<?, ?it/s]

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
import pandas as pd
newListdf = pd.DataFrame(newList,columns=list(["Yrke"]))

In [ ]:
newListdf

In [ ]:
yrkendf = pd.concat([pd.Series(row['Yrke'].split(','))              
                    for _, row in newListdf.iterrows()]).reset_index() 
yrkendf.info()

In [ ]:
mapping = {yrkendf.columns[1]:'yrke'}
yrkendf = yrkendf.rename(columns=mapping)
yrkendf["yrke"] = yrkendf['yrke'].str.strip()
yrkenAlvin = yrkendf["yrke"].value_counts() 

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)
ax = yrkenAlvin.plot(kind='pie')

ax.set_title("Vanligaste Yrken i Alvin")

In [ ]:
#fig = pie[0].get_figure()
fig = ax.get_figure()
plt.rcParams['savefig.facecolor']='white'
fig.savefig("Alvin_yrken_alla_2.png")

In [ ]:
yrkenAlvin.to_csv("Alvin_yrken_value_counts_2.csv")

In [ ]:
end_time  = datetime.now()
print("Last run: ", end_time)